In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

from IPython.display import HTML
from keras.models import Model, load_model
import numpy as np

from masterthesis.features.build_features import file_to_sequence
from masterthesis.utils import DATA_DIR, ATTENTION_LAYER, MODEL_DIR, load_split

Using TensorFlow backend.


In [3]:
def weight2color(w):
    if w < 0.5:
        ang = 240
        sat = int(50 - 100 * w)
    else:
        ang = 0
        sat = int(200 * w - 100)
    return 'hsl(%d %d%% 50%%)' % (ang, sat)

In [4]:
def show_weighted(tokens, weights):
    ents = []
    for token, weight in zip(tokens, weights):
        col = weight2color(weight)
        ent = '<span style="color: %s">%s</span>' % (col, token)
        ents.append(ent)
    display(HTML(' '.join(ents)))

In [5]:
def tokens_in_file(filepath):
    with filepath.open() as f:
        for line in f:
            for token in line.strip().split():
                yield token

In [6]:
def file_with_attention(filepath, attention_model, w2i):
    x = file_to_sequence(700, filepath, w2i)[np.newaxis, :]
    weights = attention_model.predict(x)[0]
    weights -= weights.min()
    weights /= weights.max()
    show_weighted(tokens_in_file(filepath), weights)

In [7]:
w2i = pickle.load((MODEL_DIR / 'rnn_model_w2i.pkl').open('rb'))

model = load_model(str(MODEL_DIR / 'rnn_model.h5'))

attention_model = Model(inputs=model.input, outputs=model.get_layer(ATTENTION_LAYER).output)

In [8]:
dev = load_split('dev')
dev[dev.cefr.isin(['A2', 'C1'])].head()

,age,cefr,filename,gender,lang,num_tokens,split,testlevel,title,topic
156,32,C1,h0157,kvinne,polsk,651,dev,Høyere nivå,Barn og fysiske aktiviteter,barn idrett/sport
186,51,C1,h0189,kvinne,engelsk,523,dev,Høyere nivå,Helse og livskvalitet,helse
598,31,C1,h0603,kvinne,russisk,487,dev,Høyere nivå,Adopsjon,barn familie
1375,26,A2,s0546,mann,somali,176,dev,Språkprøven,Flytting,familie flytting


In [9]:
file_with_attention(DATA_DIR / 'txt' / 'h0603.txt', attention_model, w2i)

In [10]:
file_with_attention(DATA_DIR / 'txt' / 's0546.txt', attention_model, w2i)